## Introduction


In this lab, first step is to convert addresses into their equivalent latitude and longitude values. And then to use the Foursquare API to explore venues in Aarhus City. We use the explore function to get the most common venue categories in Aarhus and search  function to get cafes 500m near to the address.We then explore trending venues near the address and find rating for all cafes to find the best one near to the address. Finally, we will use the Folium library to visualize the cafes in Aarhus City 500 m around the address highlighting the best one.

## Table of Contents

1.Download and Explore Dataset

2.Explore venues in Aarhus city

3.Search for cafes

4.Review rating for the cafes 500 m near to the address

5.Display result to the user

%%Import libraries

In [30]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 1.Download and Explore Dataset

In [31]:
CLIENT_ID = 'ISQA5CTIR5GRHAZFNKT0O015TA0J4ZUFELFQJAMRBI22HNIV' # your Foursquare ID
CLIENT_SECRET = 'PWUQ1BLLPQ3BRTJMEZJFYOHIIRMK52YV0UEHFMPTU3I24RHG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ISQA5CTIR5GRHAZFNKT0O015TA0J4ZUFELFQJAMRBI22HNIV
CLIENT_SECRET:PWUQ1BLLPQ3BRTJMEZJFYOHIIRMK52YV0UEHFMPTU3I24RHG


In [223]:
address = 'Strøget, aarhus, Denmark'
#address = 'Strøget, copenhagen, Denmark'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

56.154337 10.206346


In [224]:
LIMIT = 100# limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=ISQA5CTIR5GRHAZFNKT0O015TA0J4ZUFELFQJAMRBI22HNIV&client_secret=PWUQ1BLLPQ3BRTJMEZJFYOHIIRMK52YV0UEHFMPTU3I24RHG&v=20180605&ll=56.154337,10.206346&radius=500&limit=100'

In [225]:
results = requests.get(url).json()


In [226]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [227]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng,id
0,Pho C&P,Vietnamese Restaurant,56.152787,10.206463,574f22f3498e42c764d0abcc
1,Villa Provence,Hotel,56.154209,10.209169,4b7d5117f964a520e0b72fe3
2,Salling ROOFTOP,Roof Deck,56.154278,10.207073,599fedfead910e6c3440f9f1
3,The Burger Shack,Burger Joint,56.155265,10.205193,56cafa03498e958a8a33a017
4,Raadhuus Kafeen,Scandinavian Restaurant,56.153157,10.204232,4ba79050f964a520ca9d39e3


In [228]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,id
0,Pho C&P,Vietnamese Restaurant,56.152787,10.206463,574f22f3498e42c764d0abcc
1,Villa Provence,Hotel,56.154209,10.209169,4b7d5117f964a520e0b72fe3
2,Salling ROOFTOP,Roof Deck,56.154278,10.207073,599fedfead910e6c3440f9f1
3,The Burger Shack,Burger Joint,56.155265,10.205193,56cafa03498e958a8a33a017
4,Raadhuus Kafeen,Scandinavian Restaurant,56.153157,10.204232,4ba79050f964a520ca9d39e3


In [229]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

62 venues were returned by Foursquare.


In [230]:
no_of_venues=nearby_venues.shape[0]
no_of_venues

62

In [231]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 38 uniques categories.


In [232]:
sorted_venues = nearby_venues.pivot_table(index=['categories'], aggfunc='size')
print (sorted_venues)

categories
Art Museum                         1
Asian Restaurant                   2
Bagel Shop                         1
Bakery                             1
Bar                                2
Burger Joint                       1
Café                               6
Cocktail Bar                       3
Coffee Shop                        3
Concert Hall                       2
Electronics Store                  1
French Restaurant                  1
Gastropub                          1
Gourmet Shop                       1
Grocery Store                      1
Gym Pool                           1
Hotel                              5
Ice Cream Shop                     1
Italian Restaurant                 1
Juice Bar                          1
Market                             1
Molecular Gastronomy Restaurant    1
Multiplex                          1
Outdoor Sculpture                  1
Park                               1
Pedestrian Plaza                   1
Plaza                      

In [233]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results1 = requests.get(url).json()
results1

{'meta': {'code': 200, 'requestId': '5d8cb3f2a87921002cf648af'},
 'response': {'venues': []}}

In [234]:
if len(results1['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results1['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [235]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [236]:
search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

coffee .... OK!


In [237]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ISQA5CTIR5GRHAZFNKT0O015TA0J4ZUFELFQJAMRBI22HNIV&client_secret=PWUQ1BLLPQ3BRTJMEZJFYOHIIRMK52YV0UEHFMPTU3I24RHG&ll=56.154337,10.206346&v=20180605&query=coffee&radius=500&limit=100'

In [238]:
results = requests.get(url).json()

In [239]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b449f1ff964a52005f825e3,Banegårdspladsen 16,DK,Aarhus C,Danmark,Banegårdsgade,447,"[Banegårdspladsen 16 (Banegårdsgade), 8000 Aar...","[{'label': 'display', 'lat': 56.15074602548332...",56.150746,10.203107,NaN,8000,Region Midtjylland,Baresso Coffee,v-1569502194
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4d8dfaabd4ec8cfaf8c36689,Ryesgade 15,DK,Århus,Danmark,NaN,296,"[Ryesgade 15, 8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15179389787454...",56.151794,10.204912,NaN,8000,Region Midtjylland,Baresso Coffee,v-1569502194
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4c0fd1e881e976b07b260eeb,Frederiksgade 5,DK,Aarhus C,Danmark,Fiskergade,220,"[Frederiksgade 5 (Fiskergade), 8000 Aarhus C, ...","[{'label': 'display', 'lat': 56.15630454669505...",56.156305,10.206686,Midtbyen,8000,Region Midtjylland,Baresso Coffee,v-1569502194
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5d10bede64102800239f8c9d,NaN,DK,Århus,Danmark,NaN,153,"[8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15298, 'lng': ...",56.152980,10.205909,NaN,8000,Region Midtjylland,Coffee Ten,v-1569502194
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b449ef2f964a520fff725e3,Søndergade 2,DK,Århus,Danmark,Clemensbro,195,"[Søndergade 2 (Clemensbro), 8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15580938478404...",56.155809,10.208073,NaN,8000,Region Midtjylland,Baresso Coffee,v-1569502194


In [240]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Baresso Coffee,Coffee Shop,Banegårdspladsen 16,DK,Aarhus C,Danmark,Banegårdsgade,447,"[Banegårdspladsen 16 (Banegårdsgade), 8000 Aar...","[{'label': 'display', 'lat': 56.15074602548332...",56.150746,10.203107,NaN,8000,Region Midtjylland,4b449f1ff964a52005f825e3
1,Baresso Coffee,Coffee Shop,Ryesgade 15,DK,Århus,Danmark,NaN,296,"[Ryesgade 15, 8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15179389787454...",56.151794,10.204912,NaN,8000,Region Midtjylland,4d8dfaabd4ec8cfaf8c36689
2,Baresso Coffee,Coffee Shop,Frederiksgade 5,DK,Aarhus C,Danmark,Fiskergade,220,"[Frederiksgade 5 (Fiskergade), 8000 Aarhus C, ...","[{'label': 'display', 'lat': 56.15630454669505...",56.156305,10.206686,Midtbyen,8000,Region Midtjylland,4c0fd1e881e976b07b260eeb
3,Coffee Ten,Coffee Shop,NaN,DK,Århus,Danmark,NaN,153,"[8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15298, 'lng': ...",56.152980,10.205909,NaN,8000,Region Midtjylland,5d10bede64102800239f8c9d
4,Baresso Coffee,Coffee Shop,Søndergade 2,DK,Århus,Danmark,Clemensbro,195,"[Søndergade 2 (Clemensbro), 8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15580938478404...",56.155809,10.208073,NaN,8000,Region Midtjylland,4b449ef2f964a520fff725e3
5,Great Coffee,Coffee Shop,Klostergade 32,DK,Århus,Danmark,NaN,525,"[Klostergade 32, 8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.15904467791991...",56.159045,10.206855,NaN,8000,Region Midtjylland,4f2a5d1ee4b0582eff405f6a
6,Street Coffee,Coffee Shop,Vestergade 52,DK,Aarhus C,Danmark,NaN,455,"[Vestergade 52, 8000 Aarhus C, Danmark]","[{'label': 'display', 'lat': 56.15752413900688...",56.157524,10.201735,NaN,8000,Region Midtjylland,4f422c29e4b005b5367c91d5
7,Bill's Coffee,Coffee Shop,NaN,DK,NaN,Danmark,NaN,486,[Danmark],"[{'label': 'display', 'lat': 56.15757213158133...",56.157572,10.201061,NaN,NaN,NaN,521e1701498ee194cb68f2dc
8,La Cabra Coffee,Coffee Shop,Banegårdspladsen 1,DK,Århus,Danmark,NaN,436,"[Banegårdspladsen 1, 8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.150502, 'lng':...",56.150502,10.204907,NaN,8000,Region Midtjylland,5b702b38491be70039c0a7b1
9,Clever Coffee,Coffee Shop,NaN,DK,Århus,Danmark,NaN,388,"[8000 Århus, Danmark]","[{'label': 'display', 'lat': 56.151365, 'lng':...",56.151365,10.209630,NaN,8000,Region Midtjylland,59d6275e0a464d72f2dd0d75


In [241]:
no_of_venues=dataframe_filtered.shape[0]
no_of_venues

13

In [242]:
# define the dataframe columns
column_names = ['name','categories', 'rating', 'distance','address','Latitude', 'Longitude','id'] 
final_list= pd.DataFrame(columns=column_names)
for x in range(0,no_of_venues):
    venue_id = dataframe_filtered.id[x] 
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
         r = result['response']['venue']['rating'] 
    except:
         r ='This venue has not been rated yet.'
        
    final_list = final_list.append({'id': venue_id,
                                    'name':dataframe_filtered.name[x],
                                    'distance':dataframe_filtered.distance[x],
                                    'address':dataframe_filtered.address[x],
                                    'Latitude':dataframe_filtered.lat[x],
                                    'Longitude':dataframe_filtered.lng[x],
                                    'categories':dataframe_filtered.categories[x],
                                     'rating': r}, ignore_index=True)

In [243]:
final_list


,name,categories,rating,distance,address,Latitude,Longitude,id
0,Baresso Coffee,Coffee Shop,This venue has not been rated yet.,447,Banegårdspladsen 16,56.150746,10.203107,4b449f1ff964a52005f825e3
1,Baresso Coffee,Coffee Shop,This venue has not been rated yet.,296,Ryesgade 15,56.151794,10.204912,4d8dfaabd4ec8cfaf8c36689
2,Baresso Coffee,Coffee Shop,This venue has not been rated yet.,220,Frederiksgade 5,56.156305,10.206686,4c0fd1e881e976b07b260eeb
3,Coffee Ten,Coffee Shop,This venue has not been rated yet.,153,NaN,56.152980,10.205909,5d10bede64102800239f8c9d
4,Baresso Coffee,Coffee Shop,This venue has not been rated yet.,195,Søndergade 2,56.155809,10.208073,4b449ef2f964a520fff725e3
5,Great Coffee,Coffee Shop,This venue has not been rated yet.,525,Klostergade 32,56.159045,10.206855,4f2a5d1ee4b0582eff405f6a
6,Street Coffee,Coffee Shop,This venue has not been rated yet.,455,Vestergade 52,56.157524,10.201735,4f422c29e4b005b5367c91d5
7,Bill's Coffee,Coffee Shop,This venue has not been rated yet.,486,NaN,56.157572,10.201061,521e1701498ee194cb68f2dc
8,La Cabra Coffee,Coffee Shop,This venue has not been rated yet.,436,Banegårdspladsen 1,56.150502,10.204907,5b702b38491be70039c0a7b1
9,Clever Coffee,Coffee Shop,This venue has not been rated yet.,388,NaN,56.151365,10.209630,59d6275e0a464d72f2dd0d75


In [244]:
ff=min(final_list.distance)
ff

153

In [245]:
ff1=max(final_list.rating)
ff1

'This venue has not been rated yet.'

In [246]:
column_names = [ 'rating', 'distance','id'] 
df= pd.DataFrame(columns=column_names)
df.rating=final_list.rating
df.distance=final_list.distance
for x in range(0,no_of_venues):
    df.id[x]=x+1;

In [247]:
from sklearn.preprocessing import StandardScaler

X = df.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.36099668, -1.60356745],
       [-0.75672648, -1.33630621],
       [-1.31928913, -1.06904497],
       [-1.81523251, -0.80178373],
       [-1.50434263, -0.53452248],
       [ 0.93836361, -0.26726124],
       [ 0.4202138 ,  0.        ],
       [ 0.64968014,  0.26726124],
       [ 0.27957314,  0.53452248],
       [-0.07572959,  0.80178373],
       [ 1.20484065,  1.06904497],
       [ 0.22775816,  1.33630621],
       [ 1.38989415,  1.60356745]])

In [248]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[2 1 1 1 1 2 2 2 0 0 0 0 0]


In [251]:
df["Labels"] = labels
final_list["Labels"] = labels
df

,rating,distance,id,Labels
0,This venue has not been rated yet.,447,1,2
1,This venue has not been rated yet.,296,2,1
2,This venue has not been rated yet.,220,3,1
3,This venue has not been rated yet.,153,4,1
4,This venue has not been rated yet.,195,5,1
5,This venue has not been rated yet.,525,6,2
6,This venue has not been rated yet.,455,7,2
7,This venue has not been rated yet.,486,8,2
8,This venue has not been rated yet.,436,9,0
9,This venue has not been rated yet.,388,10,0


In [252]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters )
ys = [i + x + (i*x)**2 for i in range(num_clusters )]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_list['Latitude'], final_list['Longitude'], final_list['name'], df['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Result

In [264]:
for x in range(0,no_of_venues):
    if(final_list.Labels[x]==1): 
        if(min(final_list.distance)==final_list.distance[x]):
            print(final_list.name[x] + '- ', final_list.address[x] )

Coffee Ten-  nan


## conclusion

I here conclude  that this system recommends user Coffee Ten shop as this is nearer to the location user is now. It system could have even responded better if it has a rating.